# Realtime inference

Define a Kafka consumer to subscribe to messages in the feature store

In [ ]:
from confluent_kafka import Consumer
import json

def fetch_all_feature_records():
    # Kafka Consumer configuration for reading from the beginning of the topic
    conf = {
        'bootstrap.servers': "localhost:9092",
        'group.id': "feature_store_reader",
        'auto.offset.reset': 'latest'
        }

    # Initialize Kafka Consumer and subscribe to the topic
    consumer = Consumer(conf)
    consumer.subscribe(['feature_store'])

    feature_records = []  # List to store feature data

    try:
        while True:
            msg = consumer.poll(1.0)  # Poll for messages with a 1-second timeout
            if msg is None:
                break  # Exit loop if no more messages
            if not msg.error():
                # Convert message from JSON and add to list
                feature_records.append(json.loads(msg.value().decode('utf-8')))
            else:
                break  # Exit loop on error
    finally:
        consumer.close()  # Clean up: close consumer

    return feature_records  # Return the collected feature records

Take a look at the available messages

In [ ]:
import pandas as pd
features_df = pd.DataFrame(fetch_all_feature_records())
features_df

Get the latest complete feature record

In [ ]:
latest_feature_record = features_df.groupby('id').first().dropna().sort_index(ascending=False)[:1]
latest_feature_record

Get predictions for this record

In [ ]:
# Load model and run prediction
import joblib
def load_model(model_path):
  model = joblib.load(model_path)
  return model

model = load_model("models/energy_demand_model_v4.pkl")
feature_names = ['lag_1', 'lag_2', 'lag_6', 'lag_12', 'lag_24', 'rolling_mean_7', 'rolling_std_7', 'hour', 'day_of_week', 'month', 'temperature_forecast']
latest_feature_record = latest_feature_record[feature_names]
model.predict(latest_feature_record)